<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/focal_loss_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#connect drive with colab notebook/

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


https://github.com/keras-team/keras/issues/8130  
https://stackoverflow.com/questions/56796331/how-to-input-multiple-images-with-flow-from-dataframe-in-keras  
https://stackoverflow.com/questions/50686217/keras-how-is-accuracy-calculated-for-multi-label-classification

In [2]:
#following are libraries to unpack 7z file
!pip install patool 
!pip install pyunpack
from pyunpack import Archive

     |████████████████████████████████| 81kB 2.9MB/s 
  Created wheel for pyunpack: filename=pyunpack-0.1.2-cp36-none-any.whl size=5162 sha256=a98d046f81e9d7cb2a1b92ebed62fcaf87b4ee6522a64a3e5c636f99871ed9cd
  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack


In [0]:
import pandas as pd
import numpy as np
import cv2

In [0]:

#create a folder name as data
!mkdir data

In [0]:
#path where our data is saved
path='/content/drive/My Drive/ghina/thesis/'

In [0]:
#extract the 7z file in data folder we created above
#this not only extract the data but move it from drive to colab for efficinet processing
#it will take some time
Archive(path+'ODIR-5K_Training_Images.7z').extractall('data')

In [0]:
#read the annottion file
annotation=pd.read_excel(path+'ODIR-5K_Training_Annotations(Updated)_V2.xlsx')

In [0]:

#normal (N), diabetes (D), glaucoma (G), cataract (C), AMD (A), hypertension (H), myopia (M) and other diseases/abnormalities (O)

In [9]:
#print first 5 rows of annotation file
annotation=annotation.sample(frac=1)
annotation.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
3229,4375,57,Female,4375_left.jpg,4375_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0
1715,2338,64,Male,2338_left.jpg,2338_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
53,53,65,Female,53_left.jpg,53_right.jpg,wet age-related macular degeneration,dry age-related macular degeneration,0,0,0,0,1,0,0,0
1853,2476,70,Female,2476_left.jpg,2476_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
126,126,66,Male,126_left.jpg,126_right.jpg,dry age-related macular degeneration,dry age-related macular degeneration,0,0,0,0,1,0,0,0


In [0]:
train_annotation=annotation[0:2500]
val_annotation=annotation[2500:3500]


In [0]:
from keras_preprocessing.image import ImageDataGenerator

train_data_gen= ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest'
)

val_data_gen= ImageDataGenerator(
    rescale=1/255
    
)

In [0]:

def get_flow_from_dataframe(generator, dataframe,directory_path,
                            image_shape=(224, 224),
                            #subset='training',
                             batch_size=50):
    train_generator_1 = generator.flow_from_dataframe(dataframe,
                                                      directory=directory_path,
                                                      target_size=image_shape,
                                                      x_col="Left-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )

    train_generator_2 = generator.flow_from_dataframe(dataframe, 
                                                      directory=directory_path,
                                                      target_size=image_shape,
                                                      x_col="Right-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )
    while True:
        x_1 = train_generator_1.next()
        x_2 = train_generator_2.next()

        yield [x_1[0], x_2[0]], x_1[1]

In [0]:
batch_size=25
train_gen = get_flow_from_dataframe(train_data_gen, train_annotation, image_shape=(224, 224),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)
valid_gen = get_flow_from_dataframe(val_data_gen, val_annotation, image_shape=(224, 224),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)

In [14]:
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [29]:

import keras
left_input=Input(shape=(224, 224, 3))
right_input=Input(shape=(224, 224, 3))

left_model = ResNet50(include_top=False,input_tensor=left_input)
for layer in left_model.layers:
        layer.name = layer.name + '_left'
        layer.trainable = False


right_model = ResNet50(include_top=False,input_tensor=right_input)
for layer in right_model.layers:
        layer.name = layer.name + '_right'
        layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:


X=concatenate([left_model.output, right_model.output])
X=Flatten()(X)
X=Dropout(0.25)(X)
X=Dense(512,activation='relu')(X)
X=Dropout(0.25)(X)
out=Dense(8, activation='sigmoid')(X)

model=Model(inputs=[left_input,right_input],outputs=out)



In [0]:
import keras.backend as K
def focal_loss(y_true, y_pred, alpha = 0.75, gamma = 2):
  alpha_factor = K.ones_like(y_true) * alpha
  alpha_factor = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
  focal_weight = tf.where(K.equal(y_true, 1), 1 - y_pred, y_pred)
  focal_weight = alpha_factor * focal_weight ** gamma
  loss = focal_weight * K.binary_crossentropy(y_true, y_pred)
  return loss

In [0]:
model.compile('Adadelta',loss=focal_loss,metrics=['accuracy'])

In [0]:
step_size_train=2500/batch_size
step_size_val=1000/batch_size


In [34]:
y_true=val_annotation.iloc[:,-8::].values
y_true.shape

(1000, 8)

In [35]:
model.fit_generator(train_gen, epochs=5,
                        steps_per_epoch=step_size_train,
                        use_multiprocessing=True,
                        validation_data=valid_gen,
                        validation_steps=step_size_val,
                        verbose=2
                    
                        )

Epoch 1/5Found 1000 validated image filenames.

Found 2500 validated image filenames.
Found 1000 validated image filenames.
Found 2500 validated image filenames.

 - 440s - loss: 1.6146 - acc: 0.3228 - val_loss: 1.6323 - val_acc: 0.3200
Epoch 2/5
 - 401s - loss: 1.6246 - acc: 0.3232 - val_loss: 1.6323 - val_acc: 0.3200
Epoch 3/5
 - 401s - loss: 1.6234 - acc: 0.3232 - val_loss: 1.6323 - val_acc: 0.3200
Epoch 4/5
 - 399s - loss: 1.6160 - acc: 0.3232 - val_loss: 1.6323 - val_acc: 0.3200
Epoch 5/5
 - 397s - loss: 1.5985 - acc: 0.3232 - val_loss: 1.5909 - val_acc: 0.3200


In [36]:
y_pred = model.predict_generator(valid_gen,steps=step_size_val)


Found 1000 validated image filenames.
Found 1000 validated image filenames.


In [0]:
 model.evaluate_generator(valid_gen,steps=step_size_val,use_multiprocessing=True)

In [0]:
y_pred1=y_pred.copy()
y_pred1[y_pred1>=0.3] = 1
y_pred1[y_pred1<0.3] = 0
y_pred1=y_pred1.astype(int)

In [40]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix
#target_names = ['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']
#print(classification_report(y_true, y_pred, target_names=target_names))
multilabel_confusion_matrix(y_true, y_pred1)

array([[[678,   0],
        [322,   0]],

       [[  0, 680],
        [  0, 320]],

       [[936,   0],
        [ 64,   0]],

       [[942,   0],
        [ 58,   0]],

       [[948,   0],
        [ 52,   0]],

       [[967,   0],
        [ 33,   0]],

       [[947,   0],
        [ 53,   0]],

       [[726,   0],
        [274,   0]]])

In [41]:
print(classification_report(y_true, y_pred1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       322
           1       0.32      1.00      0.48       320
           2       0.00      0.00      0.00        64
           3       0.00      0.00      0.00        58
           4       0.00      0.00      0.00        52
           5       0.00      0.00      0.00        33
           6       0.00      0.00      0.00        53
           7       0.00      0.00      0.00       274

   micro avg       0.32      0.27      0.29      1176
   macro avg       0.04      0.12      0.06      1176
weighted avg       0.09      0.27      0.13      1176
 samples avg       0.32      0.26      0.28      1176



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_pred1[0:10]

In [0]:
np.round(y_pred[0:10],2)

In [0]:
y_true[0:10]

In [0]:
K.clear_session()